# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-27.01,96,99,3.34,RU,1738753479
1,1,ribeira grande,38.5167,-28.7000,18.22,94,75,6.26,PT,1738753480
2,2,cestos city,5.4600,-9.5828,28.75,67,12,2.26,LR,1738753482
3,3,dudinka,69.4058,86.1778,-24.91,99,95,1.39,RU,1738753483
4,4,cabo san lucas,22.8909,-109.9124,16.27,61,0,4.51,MX,1738753213


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"]>=15) & (city_data_df["Max Temp"]<=35) & (city_data_df)["Humidity"]<=60 & (city_data_df["Cloudiness"]<=60)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-27.01,96,99,3.34,RU,1738753479
1,1,ribeira grande,38.5167,-28.7000,18.22,94,75,6.26,PT,1738753480
3,3,dudinka,69.4058,86.1778,-24.91,99,95,1.39,RU,1738753483
6,6,udachny,66.4167,112.4000,-22.51,99,100,2.73,RU,1738753331
7,7,yuzhno-kurilsk,44.0306,145.8556,-2.34,76,65,4.94,RU,1738753487
...,...,...,...,...,...,...,...,...,...,...
591,591,brigantine,39.4101,-74.3646,0.59,62,100,1.54,US,1738754167
592,592,mirny,62.5353,113.9611,-40.72,100,47,1.39,RU,1738754174
593,593,mesyagutovo,55.5303,58.2528,-2.20,85,100,2.66,RU,1738754175
595,595,tidjikja,18.5564,-11.4271,22.69,14,0,8.27,MR,1738754177


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tiksi,RU,71.6872,128.8694,96,
1,ribeira grande,PT,38.5167,-28.7000,94,
3,dudinka,RU,69.4058,86.1778,99,
6,udachny,RU,66.4167,112.4000,99,
7,yuzhno-kurilsk,RU,44.0306,145.8556,76,
...,...,...,...,...,...,...
591,brigantine,US,39.4101,-74.3646,62,
592,mirny,RU,62.5353,113.9611,100,
593,mesyagutovo,RU,55.5303,58.2528,85,
595,tidjikja,MR,18.5564,-11.4271,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    # print(name_address)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tiksi - nearest hotel: Арктика
ribeira grande - nearest hotel: Hospedaria JSF
dudinka - nearest hotel: Виктория
udachny - nearest hotel: Вилюй
yuzhno-kurilsk - nearest hotel: Дом дружбы
alotau - nearest hotel: Alotau International Hotel
lae - nearest hotel: Huon Gulf Hotel
albany - nearest hotel: No hotel found
mount pearl - nearest hotel: Sandman Signature St. John's Hotel
grytviken - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
linxia chengguanzhen - nearest hotel: 如家酒店
ishinomaki - nearest hotel: 丹美旅館
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
port-aux-francais - nearest hotel: Keravel
pisco - nearest hotel: La Portada
saldanha - nearest hotel: Hoedjesbaai
longyearbyen - nearest hotel: The Vault
guerrero negro - nearest hotel: Plaza sal paraiso
port alfred - nearest hotel: No hotel found
nuku'alofa - nearest hotel: City Hotel
kailua-kona - nearest hotel: PACIFIC 19 Kona
soe - nearest hotel: DENA
fale o

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tiksi,RU,71.6872,128.8694,96,Арктика
1,ribeira grande,PT,38.5167,-28.7000,94,Hospedaria JSF
3,dudinka,RU,69.4058,86.1778,99,Виктория
6,udachny,RU,66.4167,112.4000,99,Вилюй
7,yuzhno-kurilsk,RU,44.0306,145.8556,76,Дом дружбы
...,...,...,...,...,...,...
591,brigantine,US,39.4101,-74.3646,62,La Sammana
592,mirny,RU,62.5353,113.9611,100,Зарница
593,mesyagutovo,RU,55.5303,58.2528,85,Месягутово
595,tidjikja,MR,18.5564,-11.4271,14,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points('Lng', 'Lat', 
    geo=True, tiles = True, alpha=0.70, marker='o', 
    frame_width = 700, frame_height = 500,
    s='Humidity', c='City', 
    hover_alpha=1, 
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)